In [ ]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import csv
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import hvplot.pandas
import requests

# File to Load
data_to_load = Path("Resources/Real_Estate_data.csv")

# Read Real Estate Data File and store into Pandas DataFrames
real_estate_data_df = pd.read_csv(data_to_load, low_memory=False)

# view the data
real_estate_data_df.head()

In [ ]:
#Filter Data to have just five most populated towns in CT "Bridgeport", "Stamford", "New Haven", "Hartford", "Waterbury"

narrow_df = real_estate_data_df[real_estate_data_df['Town'].isin(["Bridgeport", "Stamford", "New Haven", "Hartford", "Waterbury"])]
narrow_df.head()

In [ ]:
narrow_df['List Year'].unique()

In [ ]:
# Filter out rows where the Year is between 2001 and 2005 (inclusive)
df_filtered = narrow_df[~narrow_df['List Year'].between(2001, 2005)]

In [ ]:
df_filtered.head()

In [ ]:
df_filtered["Residential Type"].unique()

In [ ]:
df_filtered_1 = df_filtered.dropna(subset=['Residential Type'])
df_filtered_1["Residential Type"].unique()

In [ ]:
df_filtered_2 = df_filtered_1.dropna(subset=['Residential Type'])
df_filtered_2["Residential Type"].unique()

In [ ]:
property_counts = df_filtered_2["Residential Type"].value_counts()
property_counts

In [ ]:
x_axis = property_counts.index
edgecolor = 'black'
plt.bar(x_axis, property_counts, color="y", align="center", ec=edgecolor)
plt.xlim(-0.75, len(x_axis)-0.25)
plt.ylim(0, max(property_counts)+10000)
plt.title("Residential Sale Type")
plt.grid(True,linestyle=':', alpha=1)
plt.xlabel("Residential Type")
plt.ylabel("Sale Amount")

plt.show()

In [ ]:
clean_df = pd.DataFrame(df_filtered_2)
clean_df.head()

In [ ]:
# Remove rows with NaN values
clean_df = clean_df.dropna(subset=['Longitude', 'Latitude', 'Sale Amount', 'Town'])

# Ensure 'Longitude', 'Latitude', and 'Sale Amount' are numeric
clean_df['Longitude'] = pd.to_numeric(clean_df['Longitude'], errors='coerce')
clean_df['Latitude'] = pd.to_numeric(clean_df['Latitude'], errors='coerce')
clean_df['Sale Amount'] = pd.to_numeric(clean_df['Sale Amount'], errors='coerce')

# Remove rows where conversion to numeric resulted in NaN
clean_df = clean_df.dropna(subset=['Longitude', 'Latitude', 'Sale Amount'])
clean_df.head()

In [ ]:
rs_data_df = pd.DataFrame(clean_df)
rs_data_df.head()

In [ ]:
rs_data_df['Sale Amount'] = pd.to_numeric(rs_data_df['Sale Amount'], errors = 'coerce')
rs_data_res_type = rs_data_df.groupby('List Year')['Sale Amount'].sum()
rs_data_res_type

In [ ]:
#Analysis by year w/ graph

fig = plt.figure(figsize=(15,6))
x = rs_data_res_type.index
y = rs_data_res_type

bar_width = 0.5
edgecolor = 'black'

bars = plt.bar(x, y, bar_width, color="y", ec=edgecolor, lw=1)
plt.ylabel('Sale Amount')
plt.xlabel('List Year')
plt.title('House Sale Amount in CT 2006 to 2022') 
plt.grid(True,linestyle=':', alpha=0.7)
plt.xlim(2005, x.max()+1)
plt.xticks(x, rs_data_res_type.index)

for bar in bars:
        height = bar.get_height()

plt.show()

In [ ]:
#Find 6 towns in CT Bridgeport, Stamford, New Haven, Hartford, and Waterbury

In [ ]:
rs_data_df['Sale Amount'] = pd.to_numeric(rs_data_df['Sale Amount'], errors = 'coerce')
rs_data_res_sale = rs_data_df.groupby('Town')['Sale Amount'].sum()
rs_data_res_sale

In [ ]:
mean = rs_data_df['Assessed Value'].groupby(rs_data_df['Town']).mean()
median = rs_data_df['Assessed Value'].groupby(rs_data_df['Town']).median()
var = rs_data_df['Assessed Value'].groupby(rs_data_df['Town']).var()
std = rs_data_df['Assessed Value'].groupby(rs_data_df['Town']).std()
sem = rs_data_df['Assessed Value'].groupby(rs_data_df['Town']).sem()

sum_stat_av = pd.DataFrame({"Mean Assessed Value ":mean, 
                            "Median Assessed Value":median, 
                           "Assessed Value Variance":var, 
                           "Assessed Value Std. Dev.":std, 
                           "Assessed Value Std. Err.":sem})

sum_stat_av

In [ ]:
mean = rs_data_df['Sale Amount'].groupby(rs_data_df['Town']).mean()
median = rs_data_df['Sale Amount'].groupby(rs_data_df['Town']).median()
var = rs_data_df['Sale Amount'].groupby(rs_data_df['Town']).var()
std = rs_data_df['Sale Amount'].groupby(rs_data_df['Town']).std()
sem = rs_data_df['Sale Amount'].groupby(rs_data_df['Town']).sem()

sum_stat_sv = pd.DataFrame({"Mean Sale ":mean, 
                            "Median Mean Sale ":median, 
                           "Sale Amount Variance":var, 
                           "Sale Amount Std. Dev.":std, 
                           "Sale Amount":sem})

sum_stat_sv

In [ ]:
# Merge the two dataframes on the 'Town' index
merged_df = pd.merge(sum_stat_av, sum_stat_sv, left_index=True, right_index=True, suffixes=('_assessed', '_sale'))

# Plotting the grouped bar chart
fig, ax = plt.subplots(figsize=(12, 6))

# Defining the bar width
bar_width = 0.35

# Create a range for the x-axis
towns = np.arange(len(merged_df))

# Plotting the bars
ax.bar(towns - bar_width/2, merged_df['Mean Assessed Value '], width=bar_width, label='Mean Assessed Value', color='teal')
ax.bar(towns + bar_width/2, merged_df['Mean Sale '], width=bar_width, label='Mean Sale Value', color='coral')

# Add labels and title
ax.set_xlabel('Town')
ax.set_ylabel('Value')
ax.set_title('Comparison of Mean Assessed Value and Mean Sale Value by Town')
ax.set_xticks(towns)
ax.set_xticklabels(merged_df.index, rotation=90)

# Adding a legend
ax.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
#Visualize data/trend for house price

In [ ]:
#Analysis and comments/insights